In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PSG_PDF import PDF, report_data
from pylab import rcParams
from tqdm import tqdm

rcParams['figure.figsize'] = 12,4

import warnings
warnings.filterwarnings('ignore')

c:\Users\user\anaconda3\Lib\site-packages\fpdf\__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [2]:
day = '20240103'
store = f'processed file/{day}/report'

kine_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/kine/*csv'))]
force_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/force/*csv'))]
comment_path = f'comment_file/{day}/'

kine_name = [f"{os.path.basename(i).split('.')[0]}" for i in kine_list]
force_name = [f"{os.path.basename(i).split('.')[0]}" for i in force_list]

comment_path

'comment_file/20240103/'

In [3]:
kine_data = {}
for k, n in zip(kine_list, kine_name):
    kine_data[n] = k
    
force_data = {}
for f, n in zip(force_list, force_name):
    force_data[n] = f

In [4]:
kine_data.keys()

dict_keys(['ryuhanjun_0001_R_87_182_OH_135_S', 'ryuhanjun_0002_R_87_182_OH_134_S', 'ryuhanjun_0003_R_87_182_OH_137_S', 'ryuhanjun_0004_R_87_182_OH_135_S', 'ryuhanjun_0005_R_87_182_OH_134_S', 'ryuhanjun_0006_R_87_182_OH_135_S', 'ryuhanjun_0007_R_87_182_OH_134_S', 'ryuhanjun_0008_R_87_182_OH_135_S', 'ryuhanjun_0010_R_87_182_OH_135_S'])

In [6]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        self.cell(0, -10, 'Kookmin University Pitching Report', ln = True)

    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # Logo
        self.image(f'imagefile/logo.png',
                160, 280, 50)
    
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(200, 220, 255)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {
    'PELVIS_ANGLUAR_VELOCITY_Z'        : ['PELVIS'   , 'r'],
    'TORSO_ANGLUAR_VELOCITY_Z'         : ['TORSO'    , 'g'],
    'LEAD_ELBOW_ANGULAR_VELOCITY_X'    : ['ELBOW'    , 'b'],
    'LEAD_SHOULDER_ANGULAR_VELOCITY_Z' : ['SHOULDER' , 'y'],
}

ang_cols = {
    'TORSO_PELVIS_ANGLE_Z'            : 'Hip-Shoulder Separation',
    'LEAD_ELBOW_ANGLE_X'              : 'ELBOW FLEXION',
    'LEAD_SHOULDER_ANGLE_Z'           : 'SHOULDER EXTERNAL ROTATION',          
    'LEAD_SHOULDER_ANGLE_X'           : 'SHOULDER HORIZONTAL ABDUCTION',
    'LEAD_KNEE_ANGULAR_VELOCITY_X'    : 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY',
    'LEAD_SHOULDER_ANGLE_Y'           : 'SHOULDER ABDUCTION', 
    'LEAD_KNEE_ANGLE_X'               : 'LEAD LEG KNEE FLEXION',
    'TORSO_ANGLE_X'                   : 'TRUNK FORWARD TILT',
    'TORSO_ANGLE_Y'                   : 'TRUNK LATERAL TILT',
}

ap_cols = {
    'REAR_FORCE_Y' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Y' : ['Stride Leg', 'r'],
}

vt_cols = {
    'REAR_FORCE_Z' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Z' : ['Stride Leg', 'r']
}
for name in tqdm(kine_data):
    kine = kine_data[name]
    force = force_data[name]

    k_df = pd.read_csv(kine)
    f_df = pd.read_csv(force)
    

    with open(f"{comment_path}/{name.split('_')[0]}_{name.split('_')[1]}_comment.txt", "rb") as fh:
        comment_txt = fh.read().decode('utf-8')
    
    position_kinematic = comment_txt.find('- KINEMATIC SEQUENCE & STRIDE LENGTH')
    position_grf_y = comment_txt.find('- GROUND REACTION FORCE (AP axis)')
    position_grf_z = comment_txt.find('- GROUND REACTION FORCE (Vertical)')
    position_xfactor = comment_txt.find('- HIP/SHOULDER SEPARATION')
    position_elb_flx = comment_txt.find('- ELBOW FLEXION')
    position_ser = comment_txt.find('- SHOULDER EXTERNAL ROTATION')
    position_sho_ha = comment_txt.find('- SHOULDER HORIZONTAL ABDUCTION')
    position_sho_abd = comment_txt.find('- SHOULDER ABDUCTION')
    position_lead_leg_knee_ext_vel = comment_txt.find('- LEAD LEG KNEE EXTENSION ANGULAR VELOCITY')
    position_lead_leg_knee_flx = comment_txt.find('- LEAD LEG KNEE FLEXION')
    position_trunk_fwd_tilt = comment_txt.find('- TRUNK FORWARD TILT')
    position_trunk_lat_tilt = comment_txt.find('- TRUNK LATERAL TILT')
    
    kinmatic_comment = comment_txt[position_kinematic:position_grf_y]
    grf_y_comment = comment_txt[position_grf_y:position_grf_z]
    grf_z_comment = comment_txt[position_grf_z:position_xfactor]
    xfactor_comment = comment_txt[position_xfactor:position_elb_flx]
    elb_flx_comment = comment_txt[position_elb_flx:position_ser]
    ser_comment = comment_txt[position_ser:position_sho_ha]
    sho_hor_ab_comment = comment_txt[position_sho_ha:position_sho_abd]
    sho_abd_comment = comment_txt[position_sho_abd:position_lead_leg_knee_ext_vel]
    lead_leg_knee_ext_vel_comment = comment_txt[position_lead_leg_knee_ext_vel:position_lead_leg_knee_flx]
    lead_leg_knee_flx_comment = comment_txt[position_lead_leg_knee_flx:position_trunk_fwd_tilt]
    trunk_fwd_tilt_comment = comment_txt[position_trunk_fwd_tilt:position_trunk_lat_tilt]
    trunk_lat_tilt_comment = comment_txt[position_trunk_lat_tilt:]
    
    k_sr = 180
    k_kh_time = k_df['kh_time'][0]
    k_fc_time = k_df['fc_time'][0] - k_kh_time
    k_fp_time = k_df['fp_time'][0] - k_kh_time
    k_mer_time = k_df['mer_time'][0] - k_kh_time
    k_br_time = k_df['br_time'][0] - k_kh_time
    stride_length = round(k_df['stride_length'][0])
    
    f_sr = 1080
    f_kh_time = f_df['kh_time'][0] 
    f_fp_time = f_df['fp_time'][0] - f_kh_time
    f_mer_time = f_df['mer_time'][0] - f_kh_time
    f_br_time = f_df['br_time'][0] - f_kh_time

    k_df = k_df.iloc[k_kh_time:int(k_br_time + k_kh_time + (k_sr * 0.2)),:].reset_index(drop=True)
    f_df = f_df.iloc[f_kh_time:int(f_br_time + f_kh_time + (f_sr * 0.2)),:].reset_index(drop=True)

    f_lead_peak_z_time = f_df['lead_peak_z'][0] - f_kh_time
    f_rear_peak_z_time = np.where(f_df['REAR_FORCE_Z'] == f_df['REAR_FORCE_Z'].max())[0][0]
    force_peak_time = round((f_lead_peak_z_time - f_rear_peak_z_time) / 1080 , 4)

    k_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)
    f_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)

    k_len = len(k_df)
    k_time = np.arange(0,k_len/k_sr, 1/k_sr)

    f_len = len(f_df)
    f_time = np.arange(0,f_len/f_sr, 1/f_sr)

    k_fc_br = k_br_time - k_fp_time
    f_fc_br = f_br_time - f_fp_time

    y   = report_data.grf(f_df, ap_cols, f_time, f_fp_time, f_mer_time, f_br_time,f_rear_peak_z_time,f_lead_peak_z_time, axis='ap')
    z   = report_data.grf(f_df, vt_cols, f_time, f_fp_time, f_mer_time, f_br_time,f_rear_peak_z_time,f_lead_peak_z_time, axis='vt')
    ang = report_data.one_angle(k_df, ang_cols, k_time, k_fc_time, k_fp_time, k_mer_time, k_br_time)
    ks  = report_data.kinematic_sequence(k_df, ks_cols, k_time, k_fp_time, k_mer_time, k_br_time)
    report_data.kinematic_sequence_comparison_bar_individual(k_df)
    #report_data.kinematic_sequence_comparison_radar_individual(k_df)

    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)  
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)  
    # main page
    pdf.add_page()
    pdf.set_font('helvetica', 'BU', 20)
    title = 'Kookmin University Pitching Report'
    pdf.ln(5)
    title_w = pdf.get_string_width(title) + 5
    doc_w = pdf.w
    pdf.set_x((doc_w - title_w)/2)
    pdf.cell(135, 20, title, align = 'C', ln = True)
    pdf.set_fill_color(200, 220, 255)

    # ===========================================================================================================
    # chapter
    player, trial, side, weight, height, type, ballspeed, _= name.split('_')
    if side == 'R':
        side = 'Right'
    elif side == 'L':
        side = 'Left'

    if type == 'OH':
        type = 'Overhead'
    elif type == 'UD':
        type = 'Under'
    elif type == 'TQ':
        type = 'ThreeQuater'
    elif type == 'SD':
        type = 'Side'

    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()
    ch8_link = pdf.add_link()
    ch9_link = pdf.add_link()
    ch10_link = pdf.add_link()
    ch11_link = pdf.add_link()
    ch12_link = pdf.add_link()
    ch13_link = pdf.add_link()

    pdf.set_font('helvetica', 'B', 8)
    pdf.cell(0, 2,"",ln=1,align = '')
    pdf.cell(0, 0, f"Name : {player}    Date : {day}   Hand : {side}    Ball Velocity : {ballspeed} km/h   Weight : {weight}kg    Height : {height} cm", align = 'C', ln = True)
    pdf.cell(0, 10,"",ln=1,align = '')
    pdf.image(f'imagefile/analysis.png',x= 30,w=150,h=55)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,7,"",ln=1,align = '')

    pdf.cell(0, 7, 'PITCHING EFFICIENCY', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. KINEMATIC SEQUENCE', ln = 1, align = '', link = ch1_link)
    pdf.cell(0,3,"",ln=1,align = '')

    pdf.cell(0, 7, 'STRIDE', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. HIP/SHOULDER SEPARATION', ln = 1, align = '', link = ch2_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. ELBOW FLEXION', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. GROUND REACTION FORCE(TRAIL LEG)', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,3,"",ln=1,align = '')

    pdf.cell(0, 7, 'ARM COCKING', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. SHOULDER EXTERNAL ROTATION', ln = 1, align = '', link = ch2_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. SHOULDER HORIZONTAL ABDUCTION', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. LEAD LEG KNEE FLEXION', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '4. LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = 1, align = '', link = ch7_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '5. GROUND REACTION FORCE(LEAD LEG)', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')

    pdf.cell(0, 7, 'ARM ACCELERATION', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. SHOULDER ABDUCTION', ln = 1, align = '', link = ch12_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. TRUNK FORWARD TILT', ln = 1, align = '', link = ch8_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. TRUNK LATERAL TILT', ln = 1, align = '', link = ch11_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '4. LEAD LEG KNEE FLEXION', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '5. LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = 1, align = '', link = ch10_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '6. GROUND REACTION FORCE(LEAD LEG)', ln = 1, align = '', link = ch10_link)
    pdf.cell(0,2,"",ln=1,align = '')

    # ===========================================================================================================
    
    # kinematic sequence

    peak_pel = round(ks['peak']['PELVIS_ANGLUAR_VELOCITY_Z']); time_pel = ks['time']['PELVIS_ANGLUAR_VELOCITY_Z']
    peak_tor = round(ks['peak']['TORSO_ANGLUAR_VELOCITY_Z']);time_tor = ks['time']['TORSO_ANGLUAR_VELOCITY_Z']
    peak_elb = round(ks['peak']['LEAD_ELBOW_ANGULAR_VELOCITY_X']);time_elb = ks['time']['LEAD_ELBOW_ANGULAR_VELOCITY_X']
    peak_sho = round(ks['peak']['LEAD_SHOULDER_ANGULAR_VELOCITY_Z']);time_sho = ks['time']['LEAD_SHOULDER_ANGULAR_VELOCITY_Z']
    #########################################################
    total_time = k_br_time - k_fp_time
    #########################################################
    pel_time = round(100 * (time_pel - k_fp_time) / total_time)
    tor_time = round(100 * (time_tor - k_fp_time) / total_time)
    elb_time = round(100 * (time_elb - k_fp_time) / total_time)
    sho_time = round(100 * (time_sho - k_fp_time) / total_time)
    
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_elb / peak_tor,2)
    fore_gain = round(peak_sho / peak_elb,2)

    pdf.add_page()
    pdf.set_link(ch1_link, page = 2)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'KINEMATIC SEQUENCE & STRIDE LENGTH', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    #pdf.cell(0, 0, 'The order and velocity of the pelvis, trunk and arm rotation')
    pdf.image(f'figure/kinematic.png',x=65,y=19,w=140,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/kinematic.png',x=10,y=19,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/kinematic.png',x=10,y=19,w=50,h=50)

    data_as_dict = {"Segment"                 : ["Pelvic",      "Torso",       "Elbow",     "Shoulder"],
                    "<145km/h"                  : ["649 ~ 840", "987 ~ 1174","2211 ~ 2710","4331 ~ 4884"],
                    "Peak Velocity"           : [peak_pel, peak_tor, peak_elb, peak_sho],
                    "Timing [FC - BR (100%)]" : [f"{pel_time} %", f"{tor_time} %",f"{elb_time} %",f"{sho_time} %"],
                    "Speed Gain"              : [''      , tor_gain,upper_gain,fore_gain]}
    pdf.set_y(73)
    
    pdf.image(f'figure/Pelvic_kinevelo.jpg', x=30, y=117, w=70)  
    pdf.image(f'figure/Torso_kinevelo.jpg', x=110, y=117, w=70)  
    pdf.image(f'figure/Elbow_kinevelo.jpg', x=30, y=165, w=70)  
    pdf.image(f'figure/Shoulder_kinevelo.jpg', x=110, y=165, w=70) 

    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Kinematic Sequence',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [30,40,35,55,40],
                    x_start='C')
    pdf.set_text_color(150,0,0)
    pdf.set_y(67)
    pdf.set_font('helvetica', 'B', 8)
    pdf.cell(5, 7, f'Stride Length : {stride_length} % (% Height)', ln = 1, align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(220)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(225)
    pdf.multi_cell(0, 8, kinmatic_comment, ln = True, border='True')
    # ===========================================================================================================

    # GRF Y
    pdf.add_page()
    pdf.set_link(ch11_link, page = 3)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'GROUND REACTION FORCE (AP axis)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_ap.png',x=70,y=22,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Y.png',x=17,y=22,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Y.png',x=17,y=22,w=50,h=50)
    
    max_rear_y = round(y['max']['REAR_FORCE_Y'])
    max_lead_y = round(y['max']['LEAD_FORCE_Y'])
    
    data_as_dict = {
                "Segment" : ['Trail Leg','Lead Leg'],
                "Max GRF (% BW)" : [max_rear_y, max_lead_y],
                "Ideal GRF (% BW)" : ['80 ~ 106', '113 ~ 152']
                    } 

    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (AP axis)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    report_data.create_bullet_chart('GRF_YT', "Trail Leg MAX", max_rear_y, 50, 80, 106, 130, unit='')
    pdf.image(f'figure/bar/GRF_YT_bar.png', -15, 108, 235,)

    report_data.create_bullet_chart('GRF_YL', "Lead Leg MAX", max_lead_y, 80, 113, 152, 180, unit='')
    pdf.image(f'figure/bar/GRF_YL_bar.png', -15, 123, 235,)

    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, grf_y_comment, ln = True, border='True')
    # ===========================================================================================================

    # GRF Z
    pdf.add_page()
    pdf.set_link(ch12_link, page = 4)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, ' GROUND REACTION FORCE (Vertical)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_vt.png',x=70,y=22,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Z.png',x=17,y=22,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Z.png',x=17,y=22,w=50,h=50)

    max_rear_z = round(z['max']['REAR_FORCE_Z'])
    max_lead_z = round(z['max']['LEAD_FORCE_Z'])
    
    data_as_dict = {
                "Segment" : ['Trail Leg','Lead Leg','Peak Time Interval'],
                "Max GRF (% BW)" : [max_rear_z,max_lead_z],
                "Ideal GRF (% BW)" : ['128 ~ 162', '194 ~ 260'],
                }
    
    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (Vertical)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')

    pdf.set_y(75.5)
    pdf.set_x(90)
    pdf.set_text_color(150,0,0)
    pdf.set_font('helvetica', 'B', 9)
    pdf.cell(5, 7, f'Blue Vertical Line ~ Red Vertical Line Time : {force_peak_time} s', ln = 1, align = '')
    pdf.set_text_color(0,0,0)
    report_data.create_bullet_chart('GRF_ZT', "Trail Leg MAX", max_rear_z, 80, 128, 162, 210, unit='')
    pdf.image(f'figure/bar/GRF_ZT_bar.png', -15, 108, 235,)

    report_data.create_bullet_chart('GRF_ZL', "Lead Leg MAX", max_lead_z, 100, 194, 260, 300, unit='')
    pdf.image(f'figure/bar/GRF_ZL_bar.png', -15, 123, 235,)
    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, grf_z_comment, ln = True, border='True')
    # ===========================================================================================================
    
    # HIP/SHOULDER SEPARATION
    
    pdf.add_page()
    pdf.set_link(ch2_link, page = 5)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'HIP/SHOULDER SEPARATION', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    #pdf.cell(0, 0, 'The angle between the trunk and the pelvis with respect to the ground')
    pdf.image(f'figure/Hip-Shoulder Separation.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/Hip-Shoulder Separation.png',x=10,y=20,w=50,h=40)
    elif side == 'Left':
        pdf.image(f'imagefile/L/Hip-Shoulder Separation.png',x=10,y=20,w=50,h=40)

    HS_fp = round(ang['fp_time']['TORSO_PELVIS_ANGLE_Z'])
    HS_mer = round(ang['mer_time']['TORSO_PELVIS_ANGLE_Z'])
    HS_br = round(ang['br_time']['TORSO_PELVIS_ANGLE_Z'])

    data_as_dict = {
                    "at Foot Contact" : [HS_fp],
                    "at Max ER"      : [HS_mer],
                    "at Ball Release" : [HS_br]
                    }
    pdf.set_y(70)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Hip-Shoulder Separation',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    HS_max = round(ang['fp_br']['TORSO_PELVIS_ANGLE_Z'].min())  # 최댓값
    HS_max_t = round(100 * (ang['fp_br_min']['TORSO_PELVIS_ANGLE_Z'] - k_fp_time) / k_fc_br)
    
    data_as_dict = {
                    "Min"        : [HS_max],
                    "Min Timing [FC - BR (100%)]" : [f"{HS_max_t} %"],
                    }
    
    pdf.set_y(92)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')
    
    report_data.create_bullet_chart('Hip Shoulder Separation (FC)', "FC", HS_fp, -60, -39, -25, 0, unit='°')
    pdf.image(f'figure/bar//Hip Shoulder Separation (FC)_bar.png', -15, 110, 235,)
    
    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, xfactor_comment, ln = True, border='True')
    
    # ===========================================================================================================
    
    # ELBOW FLEXION
    pdf.add_page()
    pdf.set_link(ch3_link, page = 6)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'ELBOW FLEXION', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'The sagittal plane rotation of the forearm relative to the upper arm')
    pdf.image(f'figure/ELBOW FLEXION.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/ELBOW FLEXION.png',x=10,y=20,w=50,h=40)
    elif side == 'Left':
        pdf.image(f'imagefile/L/ELBOW FLEXION.png',x=10,y=20,w=50,h=40)

    EF_fp = round(ang['fp_time']['LEAD_ELBOW_ANGLE_X'])
    EF_mer = round(ang['mer_time']['LEAD_ELBOW_ANGLE_X'])
    EF_br = round(ang['br_time']['LEAD_ELBOW_ANGLE_X'])
    
    data_as_dict = {
                    "at Foot Plant" : [EF_fp],
                    "at Max ER"       : [EF_mer],
                    "at Ball Release"  : [EF_br]
                    }
    
    pdf.set_y(73)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Elbow Flexion',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    EF_max = round(ang['fp_br']['LEAD_ELBOW_ANGLE_X'].max())  # 최댓값

    EF_max_t = round(100 * (ang['fp_br_max']['LEAD_ELBOW_ANGLE_X'] - k_fp_time) / k_fc_br)
    
    data_as_dict = {
                    "Max"        : [EF_max],
                    "Max Timing [FC - BR (100%)]" : [f"{EF_max_t} %"],
                    }
    
    pdf.set_y(95)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')
    
    report_data.create_bullet_chart('Elbow Flexion (FC)', "FC", EF_fp, 50, 76, 115, 140, unit='°')
    pdf.image(f'figure/bar//Elbow Flexion (FC)_bar.png', -15, 113, 235,)

    report_data.create_bullet_chart('Elbow Flexion (MEr)', "MER", EF_mer, 50, 82, 98, 130, unit='°')
    pdf.image(f'figure/bar//Elbow Flexion (MER)_bar.png', -15, 128, 235,)
    
    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, elb_flx_comment, ln = True, border='True')
    # ===========================================================================================================

    # SHOULDER EXTERNAL ROTATION

    pdf.add_page()
    pdf.set_link(ch4_link, page = 7)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SHOULDER EXTERNAL ROTATION', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/SHOULDER EXTERNAL ROTATION.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/SHOULDER EXTERNAL ROTATION.png',x=10,y=18,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/SHOULDER EXTERNAL ROTATION.png',x=10,y=18,w=50,h=50)

    SER_fc = round(ang['fp_time']['LEAD_SHOULDER_ANGLE_Z'])
    SER_mer = round(ang['mer_time']['LEAD_SHOULDER_ANGLE_Z'])
    SER_br = round(ang['br_time']['LEAD_SHOULDER_ANGLE_Z'])
    
    data_as_dict = {
                    "at Foot Plant" : [SER_fc],
                    "at Max ER"       : [SER_mer],
                    "at Ball Release"  : [SER_br]
                    }

    pdf.set_y(73)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Shoulder External Rotation',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    SER_max = round(ang['fp_br']['LEAD_SHOULDER_ANGLE_Z'].max())  # 최댓값
    SER_max_t = round(100 * (ang['fp_br_max']['LEAD_SHOULDER_ANGLE_Z'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "Max"        : [SER_max],
                    "Max Timing [FC - BR](100%)]" : [f"{SER_max_t} %"],
                    }
    
    pdf.set_y(95)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    report_data.create_bullet_chart('Shoulder External Rotation (FC)', "FC", SER_fc, 0, 11, 48, 90, unit='°')
    pdf.image(f'figure/bar//Shoulder External Rotation (FC)_bar.png', -15, 113, 235,)

    report_data.create_bullet_chart('Shoulder External Rotation (MER)', "MER", SER_mer, 120, 163, 178, 250, unit='°')
    pdf.image(f'figure/bar//Shoulder External Rotation (MER)_bar.png', -15, 128, 235,)

    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, ser_comment, ln = True, border='True')
    # ===========================================================================================================
    
    # SHOULDER HORIZONTAL ABDUCTION
    pdf.add_page()
    pdf.set_link(ch5_link, page = 8)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SHOULDER HORIZONTAL ABDUCTION', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/SHOULDER HORIZONTAL ABDUCTION.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/SHOULDER HORIZONTAL ABDUCTION.png',x=10,y=18,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/SHOULDER HORIZONTAL ABDUCTION.png',x=10,y=18,w=50,h=50)

    SHA_fp = round(ang['fp_time']['LEAD_SHOULDER_ANGLE_X'])
    SHA_mer = round(ang['mer_time']['LEAD_SHOULDER_ANGLE_X'])
    SHA_br = round(ang['br_time']['LEAD_SHOULDER_ANGLE_X'])
    
    data_as_dict = {
                    "at Foot Plant" : [SHA_fp],
                    "at Max ER"       : [SHA_mer],
                    "at Ball Release"  : [SHA_br]
                    }

    pdf.set_y(73)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Shoulder Horizontal Abduction',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    SHA_max = round(ang['fp_br']['LEAD_SHOULDER_ANGLE_X'].max())  # 최댓값
    SHA_max_t = round(100 * (ang['fp_br_max']['LEAD_SHOULDER_ANGLE_X'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "Max"        : [SHA_max],
                    "Max Timing [FC - BR (100%)]" : [f"{SHA_max_t} %"],
                    }
    
    pdf.set_y(96)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    report_data.create_bullet_chart('Shoulder Horizontal Abduction (FC)', "FC", SHA_fp, -100, -63, -24, 10, unit='°')
    pdf.image(f'figure/bar//Shoulder Horizontal Abduction (FC)_bar.png', -15, 113, 235,)

    report_data.create_bullet_chart('Shoulder Horizontal Abduction (MER)', "MER", SHA_mer, -90, -12, 9, 30, unit='°')
    pdf.image(f'figure/bar//Shoulder Horizontal Abduction (MER)_bar.png', -15, 128, 235,)

    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, sho_abd_comment, ln = True, border='True')
    # ===========================================================================================================
    
    # SHOULDER ABDUCTION

    pdf.add_page()
    pdf.set_link(ch6_link, page = 9)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SHOULDER ABDUCTION', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/SHOULDER ABDUCTION.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R//SHOULDER ABDUCTION.png',x=10,y=21,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L//SHOULDER ABDUCTION.png',x=10,y=21,w=50,h=50)
    
    SAB_fp = round(ang['fp_time']['LEAD_SHOULDER_ANGLE_Y'])
    SAB_mer = round(ang['mer_time']['LEAD_SHOULDER_ANGLE_Y'])
    SAB_br = round(ang['br_time']['LEAD_SHOULDER_ANGLE_Y'])

    data_as_dict = {
                    "at Foot Plant" : [SAB_fp],
                    "at Max ER"       : [SAB_mer],
                    "at Ball Release"  : [SAB_br]
                    }

    pdf.set_y(72)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Shoulder Abduction',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    SAB_max = round(ang['fp_br']['LEAD_SHOULDER_ANGLE_Y'].max())  # 최댓값
    SAB_max_t = round(100 * (ang['fp_br_max']['LEAD_SHOULDER_ANGLE_Y'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "Max"        : [SAB_max],
                    "Max Timing [FC - BR (100%)]" : [f"{SAB_max_t} %"],
                    }
    
    pdf.set_y(95)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')
    
    report_data.create_bullet_chart('Shoulder Abduction (FC)', "FC", SAB_fp, 40, 74, 97, 120, unit='°')
    pdf.image(f'figure/bar//Shoulder Abduction (FC)_bar.png', -15, 113, 235,)

    report_data.create_bullet_chart('Shoulder Abduction (BR)', "BR", SAB_br, 50, 81, 99, 120, unit='°')
    pdf.image(f'figure/bar//Shoulder Abduction (BR)_bar.png', -15, 128, 235,)

    pdf.set_y(155)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(160)
    pdf.multi_cell(0, 8, sho_abd_comment, ln = True, border='True')
    # ===========================================================================================================
    
    # LEAD LEG KNEE FLEXION
    
    pdf.add_page()
    pdf.set_link(ch7_link, page = 10)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE FLEXION', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/LEAD LEG KNEE FLEXION.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE FLEXION.png',x=10,y=21,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE FLEXION.png',x=10,y=21,w=50,h=50)
    
    LKF_fp = round(ang['fp_time']['LEAD_KNEE_ANGLE_X'])
    LKF_mer = round(ang['mer_time']['LEAD_KNEE_ANGLE_X'])
    LKF_br = round(ang['br_time']['LEAD_KNEE_ANGLE_X'])

    data_as_dict = {
                    "at Foot Plant" : [LKF_fp],
                    "at Max ER"       : [LKF_mer],
                    "at Ball Release"  : [LKF_br]
                    }

    pdf.set_y(72)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Extension Angular Velocity',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    LKF_max = round(ang['fp_br']['LEAD_KNEE_ANGLE_X'].max())  # 최댓값
    LKF_max_t = round(100 * (ang['fp_br_max']['LEAD_KNEE_ANGLE_X'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "Max"        : [LKF_max],
                    "Max Timing [FC - BR (100%)]" : [f"{LKF_max_t} %"],
                    }
    
    pdf.set_y(95)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')
    
    report_data.create_bullet_chart('Lead Leg Knee Flexion (FC)', "FC", LKF_fp, 20, 39, 66, 100, unit='°')
    pdf.image(f'figure/bar/Lead Leg Knee Flexion (FC)_bar.png', -15, 111, 235,)

    report_data.create_bullet_chart('Lead Leg Knee Flexion (BR)', "BR", LKF_br, 0, 25, 53, 90, unit='°')
    pdf.image(f'figure/bar/Lead Leg Knee Flexion (BR)_bar.png', -15, 128, 235,)

    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, lead_leg_knee_flx_comment, ln = True, border='True')
    # ===========================================================================================================
    
    # LEAD LEG KNEE EXTENSION ANGULAR VELOCITY

    pdf.add_page()
    pdf.set_link(ch8_link, page = 11)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=70,y=22,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=22,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=22,w=50,h=50)

    KEAV_fp = round(ang['fp_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])
    KEAV_mer = round(ang['mer_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])
    KEAV_br = round(ang['br_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])
    
    data_as_dict = {
                    "at Foot Plant" : [KEAV_fp],
                    "at Max ER"       : [KEAV_mer],
                    "at Ball Release"  : [KEAV_br]
                    }

    pdf.set_y(72)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Flexion',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')

    KEAV_max = round(ang['fp_br']['LEAD_KNEE_ANGULAR_VELOCITY_X'].max())  # 최댓값
    KEAV_max_t = round(100 * (ang['fp_br_max']['LEAD_KNEE_ANGULAR_VELOCITY_X'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "Max"        : [KEAV_max],
                    "Max Timing [FC - BR (100%)]" : [f"{KEAV_max_t} %"],
                    }
    
    pdf.set_y(94)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')
    
    report_data.create_bullet_chart('Lead Leg Knee Extension Angular Velocity (MAX)', "MAX", KEAV_max, 0, 218, 502, 800, unit='°/s')
    pdf.image(f'figure/bar/Lead Leg Knee Extension Angular Velocity (MAX)_bar.png', -15, 111, 235,)

    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, lead_leg_knee_ext_vel_comment, ln = True, border='True')
    # ===========================================================================================================
    
    # TRUNK FORWARD TILT
    pdf.add_page()
    pdf.set_link(ch9_link, page = 12)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRUNK FORWARD TILT', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/TRUNK FORWARD TILT.png',x=70,y=22,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/TRUNK FORWARD TILT.png',x=10,y=22,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/TRUNK FORWARD TILT.png',x=10,y=22,w=50,h=50)
    
    TFT_fp = round(ang['fp_time']['TORSO_ANGLE_X'])  ###
    TFT_mer = round(ang['mer_time']['TORSO_ANGLE_X'])
    TFT_br = round(ang['br_time']['TORSO_ANGLE_X'])

    data_as_dict = {
                    "at Foot Plant" : [TFT_fp],
                    "at Max ER"       : [TFT_mer],
                    "at Ball Release"  : [TFT_br]
                    }

    pdf.set_y(72)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Trunk Forward Tilt',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    TFT_max = round(ang['fp_br']['TORSO_ANGLE_X'].max())  # 최댓값
    TFT_max_t = round(100 * (ang['fp_br_max']['TORSO_ANGLE_X'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "Max"        : [TFT_max],
                    "Max Timing [FC - BR (100%)]" : [f"{TFT_max_t} %"],
                    }
    
    pdf.set_y(94)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')
    
    report_data.create_bullet_chart('Trunk Forward Tilt (BR)', "BR", TFT_br, 0, 22, 43, 100, unit='°')
    pdf.image(f'figure/bar/Trunk Forward Tilt (BR)_bar.png', -15, 111, 235,)

    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, trunk_fwd_tilt_comment, ln = True, border='True')
    # ===========================================================================================================

    # TRUNK LATERAL TILT

    pdf.add_page()
    pdf.set_link(ch10_link, page = 13)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRUNK LATERAL TILT', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/TRUNK LATERAL TILT.png',x=70,y=21,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/TRUNK LATERAL TILT.png',x=10,y=21,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/TRUNK LATERAL TILT.png',x=10,y=21,w=50,h=50)
    
    TLT_fp = round(ang['fp_time']['TORSO_ANGLE_Y'])
    TLT_mer = round(ang['mer_time']['TORSO_ANGLE_Y'])
    TLT_br = round(ang['br_time']['TORSO_ANGLE_Y'])

    data_as_dict = {
                    "at Foot Plant" : [TLT_fp],
                    "at Max ER"       : [TLT_mer],
                    "at Ball Release"  : [TLT_br]
                    }

    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Trunk Forward Tilt',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    TLT_max = round(ang['fp_br']['TORSO_ANGLE_Y'].max())  # 최댓값
    TLT_max_t = round(100 * (ang['fp_br_max']['TORSO_ANGLE_Y'] - k_fp_time) / k_fc_br)
    
    data_as_dict = {
                    "Max"        : [TLT_max],
                    "Max Timing [FC - BR (100%)]" : [f"{TLT_max_t} %"],
                    }
    
    pdf.set_y(97)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')
    
    report_data.create_bullet_chart('Trunk Lateral Tilt (BR)', "BR", TLT_br, -20, 9, 23, 50, unit='°')
    pdf.image(f'figure/bar/Trunk Lateral Tilt (BR)_bar.png', -15, 115, 235,)

    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, trunk_lat_tilt_comment, ln = True, border='True')
    # ===========================================================================================================

    # Summary
    pdf.add_page()
    pdf.set_link(ch13_link, page = 13)
    pdf.set_font('helvetica', 'B', 8)
    pdf.set_y(15)
    # Add Summary Title
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Summary', ln=True, align='C')
    
    # Kinematic Sequence
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(30)
    pdf.cell(190, 10, 'Kinematic Sequence', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Pelvic Angular Velocity(MAX)': '649 ~ 840°/s',
        'Torso Angular Velocity(MAX)': '987 ~ 1174°/s',
        'Elbow Angular Velocity(MAX)': '2211 ~ 2710°/s',
        'Shoulder Angular Velocity(MAX)': '4331 ~ 4884°/s'
    }
    
    mine_val = [peak_pel, peak_tor, peak_elb, peak_sho]

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(55)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
    
    # GRF
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(100)
    pdf.cell(190, 10, 'GROUND REACTION FORCE', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Trail Leg Y': '80 ~ 106',
        'Trail Leg Z': '128 ~ 162',
        'Lead Leg Y': '113 ~ 152',
        'Lead Leg Z': '194 ~ 260'
    }

    mine_val = [
                max_rear_y, max_rear_z,
                max_lead_y, max_lead_z
    ]

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(115)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
    
    # ===========================================================================================================
    pdf.add_page()
    pdf.set_y(10)
    # Add Summary Title
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Summary', ln=True, align='C')
    
    
    # Foot Contact
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(30)
    pdf.cell(190, 10, 'Foot Contact', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Elbow Flexion': '76 ~ 115°',
        'Shoulder External Rotation': '11 ~ 48°',
        'Shoulder Abduction': '74 ~ 97°',
        'Shoulder Horizontal Abduction': '-63 ~ -24°',
        'Trunk Forward Tilt': '-12 ~ 9°',
        'Lead Leg Knee Flexion': '39 ~ 66°',
    }
    
    mine_val = [
                EF_fp, SER_fc, SAB_fp,
                SHA_fp, TFT_fp, LKF_fp
    ]

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(45)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
    
    # Arm Cocking
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(110)
    pdf.cell(190, 10, 'Arm Cocking', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Elbow Flexion': '82 ~ 98°',
        'Shoulder Horizontal Abduction': '-12 ~ 9°',
        'Shoulder External Rotation': '163 ~ 178°'
    }

    mine_val = [
                EF_mer, SHA_mer, SER_mer
    ]

    pdf.set_font('helvetica', 'B', 15)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(125)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)

    # Ball Release
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(200)
    pdf.cell(190, 10, 'Ball Release', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Lead Leg Knee Flexion': '25 ~ 53°',
        'Elbow Flexion': '27 ~ 28°',
        'Shoulder Abduction': '81 ~ 99°',
        'Trunk Forward Tilt': '22 ~ 43°',
        'Trunk Lateral Tilt': '9 ~ 23°'
    }

    mine_val = [
                LKF_br, EF_br, SAB_br,
                TFT_br, TLT_br
    ]
        

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(215)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)

    figure_file = glob('figure/*png')
    for figure in figure_file:
        os.remove(figure)
        
    pdf.output(f'{store}/{day}_{player}_{trial}_{side}_baseball_report.pdf')

  0%|          | 0/9 [00:06<?, ?it/s]


KeyError: 'fp_br'